In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
import os
import keras
from keras.models import Model
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.layers import GlobalAveragePooling2D, Dense
from keras.optimizers import Adam

Using TensorFlow backend.


In [4]:
# train data
train_dir = '/content/drive/My Drive/project_cnn/driver_detection/data/train'
# images shape
img_width, img_height = 224, 224
# tensor shape
input_shape = (img_width, img_height, 3)
# number epoch
epochs = 40
# mini-batch size
batch_size = 16
# к-ть зображень для навчання
nb_train_samples=0
for i in range(10):
    nb_train_samples+=len(os.listdir(os.path.join(train_dir,"c"+str(i))))   #сума зображень в 10ти папках
print(nb_train_samples)

22442


In [5]:
model_resnet50=keras.applications.resnet.ResNet50(include_top=False,
                                               weights="imagenet",
                                               input_shape=input_shape)








94773248/94765736 [==============================] - 1s 0us/step


In [0]:
# add a global spatial average pooling layer
x = model_resnet50.output
x = GlobalAveragePooling2D()(x)

predictions = Dense(10, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=model_vgg16.input, outputs=predictions)

In [7]:
optimizer=Adam(lr=0.00001)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])


In [8]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)                                            #validation_split=0.2, # set validation split
                                   #preprocessing_function=preprocess_input)


train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')
    #subset='training') # set as training data

val_generator = train_datagen.flow_from_directory(
    "/content/drive/My Drive/project_cnn/driver_detection/data/test_min",
    target_size=(img_width, img_height),
    batch_size=100,
    class_mode='categorical')
    #subset='validation') # set as training data

Found 22434 images belonging to 10 classes.
Found 100 images belonging to 10 classes.


In [0]:
reducelronplateau=ReduceLROnPlateau(monitor='val_loss',         #callback which reduces the speed of training if the losses do not decrease
                                    factor=0.1,
                                    patience=10)

callbacks=[ModelCheckpoint("/content/drive/My Drive/project_cnn/driver_detection/saved_models_vgg16/drivedetvgg16-dense-{epoch:02d}-{val_acc:.4f}.hdf5"),reducelronplateau]

In [11]:
model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=val_generator.samples // 100,
    callbacks=callbacks)


Epoch 1/40
 177/1402 [==>...........................] - ETA: 1:42:34 - loss: 1.4650 - acc: 0.6162

KeyboardInterrupt: ignored